In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import sys
import time
import torch

from scipy.integrate import solve_ivp

PROJECT_ROOT = os.path.abspath(
    os.path.join(os.getcwd(), os.pardir)
)
sys.path.append(PROJECT_ROOT)

import plot
import sample_points
from tcpinn import TcPINN

np.random.seed(1)

## Overview

This jupyter notebook implements the plain time-consistent physics-informed neural network (tcPINN) idea for the Lorenz system.

The system reads

\begin{align*}
    \frac{d}{dt} \begin{pmatrix} x \\ y \\ z \end{pmatrix} = \begin{pmatrix} 
    \sigma (y-x) \\ x(\rho -z) - y \\ xy - \beta z\end{pmatrix}.
\end{align*}

We set $\sigma = 10, \beta = \frac{8}{3}$ and $\rho = 28$, which is a choice of parameters known to exhibit chaotic behavior.

We normalize the attractor of the ODE system by shifting $x, y, z$ by $c_x, c_y$ and $c_z$ respectively and scaling all three components by $\lambda$. The normalized Lorenz system reads

\begin{align*}
    \frac{d}{dt} \begin{pmatrix} x \\ y \\ z \end{pmatrix} = 
    \begin{pmatrix} 
    \sigma \big((y + \frac{c_y}{\lambda}) - (x + \frac{c_x}{\lambda})\big) \\
    \lambda (x + \frac{c_x}{\lambda}) (\frac{\rho}{\lambda} - (z + \frac{c_z}{\lambda})) - (y + \frac{c_y}{\lambda}) \\
    \lambda (x + \frac{c_x}{\lambda})(y + \frac{c_y}{\lambda}) - \beta (z + \frac{c_z}{\lambda}) 
    \end{pmatrix}.
\end{align*}

In [2]:
sigma = 10.
beta = 8 / 3
rho = 28.

# Note: With this normalization, the attractor is centered around the origin
shift_x = - 0.013895
shift_y = - 0.013791
shift_z = 23.584181

scaling = 8.533635

In [3]:
class LorenzSystem(TcPINN):
    """
    A tcPINN implementation of the normalized Lorenz system.
    """
    def __init__(
        self, layers, T, X_pinn=None, X_semigroup=None, X_smooth=None, X_data=None, data=None,
        w_pinn=1., w_semigroup=1., w_smooth=1., w_data=1.
    ):
        super().__init__(
            layers, T, X_pinn, X_semigroup, X_smooth, X_data, data,
            w_pinn, w_semigroup, w_smooth, w_data
        )
    
    
    def _loss_pinn(self):
        """
        The PINN loss for the normalized Lorenz system.
        """
        y = self.net_y(self.t_pinn, self.y_pinn)
        deriv = self.net_derivative(self.t_pinn, self.y_pinn)
        
        loss1 = torch.mean(
            (deriv[0] - sigma * ((y[:,1:2] + shift_y / scaling) - (y[:,0:1] + shift_x / scaling))) ** 2
        )
        loss2 = torch.mean(
            (deriv[1] - scaling * (y[:,0:1] + shift_x / scaling) * (rho / scaling - (y[:,2:3] + shift_z / scaling)) + (y[:,1:2] + shift_y / scaling)) ** 2
        )
        loss3 = torch.mean(
            (deriv[2] - scaling * (y[:,0:1] + shift_x / scaling) * (y[:,1:2] + shift_y / scaling) + beta * (y[:,2:3] + shift_z / scaling)) ** 2
        )        
        loss = self.w_pinn * (loss1 + loss2 + loss3)
        
        return loss

### Setup Training Data

In [4]:
ode_dimension = 3
layers = [ode_dimension + 1] + 4 * [128] + [ode_dimension]
T = .1

# training samples
n_pinn = 10
t_pinn = np.random.uniform(0, T, (n_pinn, 1))
y_pinn = np.random.uniform(-1, 1 , (n_pinn, ode_dimension))
X_pinn = np.hstack([t_pinn, y_pinn])

n_semigroup = 10
st_semigroup = sample_points.uniform_triangle_2d(n_semigroup, T)
y_semigroup = np.random.uniform(-1, 1 , (n_semigroup, ode_dimension))
X_semigroup = np.hstack([st_semigroup, y_semigroup])

n_smooth = 10
t_smooth = np.random.uniform(0, T, (n_smooth, 1))
y_smooth = np.random.uniform(-1, 1 , (n_smooth, ode_dimension))
X_smooth = np.hstack([t_smooth, y_smooth])

In [5]:
#ax = plot.plot_scatter_3d(y_pinn, s=1)
#plt.show()

In [6]:
model = LorenzSystem(layers, T, X_pinn=X_pinn, X_semigroup=X_semigroup, X_smooth=X_smooth)

In [7]:
%%time
model.train()

CPU times: user 26.3 s, sys: 45.8 ms, total: 26.3 s
Wall time: 3.94 s


In [8]:
path = os.getcwd()

with open(f"{path}/model_lorenz.pkl", "wb") as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(f"{path}/model_lorenz.pkl", "rb") as f:
    model = pickle.load(f)

## Predict and Plot the Solution

In [9]:
def rhs_lorenz_system(t, r, shift_x, shift_y, shift_z, scaling):
    """
    Rhs of the normalized Lorenz system
    """
    x, y, z = r
    
    fx = sigma * ((y + shift_y / scaling) - (x + shift_x / scaling))
    fy = scaling * (x + shift_x / scaling) * (rho / scaling - (z + shift_z / scaling)) - (y + shift_y / scaling)
    fz = scaling * (x + shift_x / scaling) * (y + shift_y / scaling) - beta * (z + shift_z / scaling)
    
    return np.array([fx, fy, fz])


def get_solution(max_t, delta_t, init_val):

    times = np.linspace(0, max_t, int(max_t / delta_t) + 1)
    sol = solve_ivp(
        rhs_lorenz_system, [0, float(max_t)], init_val, t_eval=times,
        rtol=1e-10, atol=1e-10, args=(shift_x, shift_y, shift_z, scaling)
    )
    return sol.y.T

In [10]:
# Note that max_t in training is .1
init_val = np.random.uniform(-1, 1 , 3)

max_t = 10.
delta_t = 0.01

true_solution = get_solution(max_t, delta_t, init_val)
tc_solution = model.predict_tc(max_t, delta_t, init_val)

In [12]:
ax = plot.plot_solution_3d(true_solution, label="truth")
ax = plot.plot_solution_3d(tc_solution, ax=ax, label="tcPINN")

plt.legend()
plt.show()
plt.close()